In [109]:
54

54

In [110]:
print("Hello ")

Hello 


In [111]:
import os

In [112]:
%pwd

'c:\\AI\\ChatBot\\E-commerce-chatbot'

In [113]:
os.chdir("../")

In [114]:
%pwd

'c:\\AI\\ChatBot'

In [115]:
DB_Name = os.getenv("DB_NAME")
DB_User = os.getenv("DB_USER")
DB_Host = os.getenv("DB_HOST")
DB_Port = os.getenv("DB_PORT")
DB_Password = os.getenv("DB_PASSWORD")

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")



In [116]:
import psycopg2

conn = psycopg2.connect(
    host=DB_Host,
    port=DB_Port,
    database=DB_Name,
    user=DB_User,
    password=DB_Password
)

cursor = conn.cursor()
cursor.execute("""
SELECT p.id, p.name, p.description, p.brand, c.name AS category,
    (SELECT json_agg(json_build_object('rating', r.rating, 'text', r.review_text, 'reviewer', r.reviewer_name))
     FROM reviews r WHERE r.product_id = p.id) as reviews
FROM products p
JOIN categories c ON p.category_id = c.id
""")
products = cursor.fetchall()


In [117]:
products

[(2,
  'Samsung Galaxy S23',
  'Flagship Android phone with great performance and display.',
  'Samsung',
  'Smartphones',
  [{'rating': 4.6,
    'text': 'Solid Android experience. Fast and sleek.',
    'reviewer': 'Raj'}]),
 (1,
  'iPhone 14 Pro',
  'Apple smartphone with A16 Bionic chip and 48MP camera.',
  'Apple',
  'Smartphones',
  [{'rating': 4.8,
    'text': 'Amazing camera and super smooth performance!',
    'reviewer': 'John Doe'},
   {'rating': 4.5,
    'text': 'Battery life could be better but love the display.',
    'reviewer': 'Emily'}]),
 (4,
  'Dell XPS 13',
  'Compact Windows laptop with 11th-gen Intel i7.',
  'Dell',
  'Laptops',
  [{'rating': 4.3,
    'text': 'Great build, but fan gets noisy under load.',
    'reviewer': 'Liam'}]),
 (3,
  'MacBook Air M2',
  'Lightweight Apple laptop with M2 chip.',
  'Apple',
  'Laptops',
  [{'rating': 4.9,
    'text': 'Perfect for work and study. Super fast and lightweight.',
    'reviewer': 'Sophia'}]),
 (5,
  'Sony WH-1000XM5',
  

In [118]:
# print(PINECONE_API_KEY)
import json

In [119]:
def prepare_text(product):
    product_id, name, description, brand, category, reviews = product
    text = f"Product: {name}\nDescription: {description}\nBrand: {brand}\nCategory: {category}\n"
    if reviews:  # If there are reviews, add them to the text
        for review in reviews:
            text += f"Review ({review['rating']} stars by {review['reviewer']}): {review['text']}\n"
    return text


In [120]:
# Loop through all products and prepare text for each
prepared_products = [prepare_text(product) for product in products]

# Print the prepared texts (optional)
# for text in prepared_products:
#     print(text)
#     print("-" * 50) 

print(prepared_products) # Just to separate them for readability


['Product: Samsung Galaxy S23\nDescription: Flagship Android phone with great performance and display.\nBrand: Samsung\nCategory: Smartphones\nReview (4.6 stars by Raj): Solid Android experience. Fast and sleek.\n', 'Product: iPhone 14 Pro\nDescription: Apple smartphone with A16 Bionic chip and 48MP camera.\nBrand: Apple\nCategory: Smartphones\nReview (4.8 stars by John Doe): Amazing camera and super smooth performance!\nReview (4.5 stars by Emily): Battery life could be better but love the display.\n', 'Product: Dell XPS 13\nDescription: Compact Windows laptop with 11th-gen Intel i7.\nBrand: Dell\nCategory: Laptops\nReview (4.3 stars by Liam): Great build, but fan gets noisy under load.\n', 'Product: MacBook Air M2\nDescription: Lightweight Apple laptop with M2 chip.\nBrand: Apple\nCategory: Laptops\nReview (4.9 stars by Sophia): Perfect for work and study. Super fast and lightweight.\n', 'Product: Sony WH-1000XM5\nDescription: Noise-cancelling over-ear headphones.\nBrand: Sony\nCateg

In [124]:
from sentence_transformers import SentenceTransformer
import pinecone
import uuid

In [125]:
# Load the sentence-transformers model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [131]:
from pinecone import Pinecone
from pinecone import ServerlessSpec

# Initialize Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)

# Define index name and parameters
index_name = 'product-embeddings'
dimension = 384  # Matches all-MiniLM-L6-v2 embedding size
metric = 'cosine'  # Common metric for text embeddings

# Check if index exists; create it if it doesn't
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric=metric,
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'  # Adjust region as needed, e.g., 'us-west1-gcp'
        )
    )

# Connect to the index
index = pc.Index(index_name)

In [133]:
# Your products list
products = [
    (2, 'Samsung Galaxy S23', 'Flagship Android phone with great performance and display.', 'Samsung', 'Smartphones', [{'rating': 4.6, 'text': 'Solid Android experience. Fast and sleek.', 'reviewer': 'Raj'}]),
    (1, 'iPhone 14 Pro', 'Apple smartphone with A16 Bionic chip and 48MP camera.', 'Apple', 'Smartphones', [{'rating': 4.8, 'text': 'Amazing camera and super smooth performance!', 'reviewer': 'John Doe'}, {'rating': 4.5, 'text': 'Battery life could be better but love the display.', 'reviewer': 'Emily'}]),
    (4, 'Dell XPS 13', 'Compact Windows laptop with 11th-gen Intel i7.', 'Dell', 'Laptops', [{'rating': 4.3, 'text': 'Great build, but fan gets noisy under load.', 'reviewer': 'Liam'}]),
    (3, 'MacBook Air M2', 'Lightweight Apple laptop with M2 chip.', 'Apple', 'Laptops', [{'rating': 4.9, 'text': 'Perfect for work and study. Super fast and lightweight.', 'reviewer': 'Sophia'}]),
    (5, 'Sony WH-1000XM5', 'Noise-cancelling over-ear headphones.', 'Sony', 'Headphones', [{'rating': 4.7, 'text': 'Incredible noise cancellation, great sound quality.', 'reviewer': 'Noah'}]),
    (6, 'Apple Watch Series 8', 'Smartwatch with health tracking and fitness features.', 'Apple', 'Smartwatches', [{'rating': 4.2, 'text': 'Good fitness features, but battery doesn’t last long.', 'reviewer': 'Mia'}])
]

# Prepare data for Pinecone
vectors_to_upsert = []
for product in products:
    product_id, name, description, brand, category, reviews = product
    
    # Embed the description (or optionally description + reviews)
    text_to_embed = description  # Use description only
    # Optional: Include reviews in embedding
    # review_texts = ' '.join([review['text'] for review in reviews])
    # text_to_embed = f"{description} {review_texts}"
    
    # Generate embedding
    embedding = model.encode(text_to_embed).tolist()
    
    # Prepare metadata, serializing reviews to JSON string
    metadata = {
        'name': name,
        'description': description,
        'brand': brand,
        'category': category,
        'reviews': json.dumps(reviews)  # Convert reviews to JSON string
    }
    
    # Prepare the vector tuple: (id, embedding, metadata)
    vectors_to_upsert.append((str(product_id), embedding, metadata))

# Upsert vectors to Pinecone
index.upsert(vectors=vectors_to_upsert)
print(f"Successfully uploaded {len(vectors_to_upsert)} products to Pinecone.")

Successfully uploaded 6 products to Pinecone.


In [134]:
# Check index stats
index_stats = index.describe_index_stats()
print(index_stats)

{'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}


In [136]:
import json

# Example: Fetch a vector by ID
vector_id = "1"  # Example product ID (e.g., iPhone 14 Pro)
fetch_response = index.fetch(ids=[vector_id])

# Access the vectors attribute of FetchResponse
vector_data = fetch_response.vectors.get(vector_id)

if vector_data:
    metadata = vector_data.metadata  # Access metadata attribute
    print("Product:", metadata['name'])
    print("Description:", metadata['description'])
    print("Brand:", metadata['brand'])
    print("Category:", metadata['category'])
    # Deserialize reviews from JSON string
    reviews = json.loads(metadata['reviews'])
    print("Reviews:", reviews)
else:
    print(f"No vector found for ID: {vector_id}")

Product: iPhone 14 Pro
Description: Apple smartphone with A16 Bionic chip and 48MP camera.
Brand: Apple
Category: Smartphones
Reviews: [{'rating': 4.8, 'text': 'Amazing camera and super smooth performance!', 'reviewer': 'John Doe'}, {'rating': 4.5, 'text': 'Battery life could be better but love the display.', 'reviewer': 'Emily'}]


In [137]:
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
import json
import os